# automatically open and scrape timereports
1. open markndown file in specific folder. Or drop the files? or have a file slection popup?
2. scrape all lines after `## Day Planner`
3. send to openai for formatting
4. use gsheets

In [1]:
# use todays date to find the file

In [2]:
from datetime import datetime
from dateutil.relativedelta import relativedelta
from pathlib import Path
import pathlib
import numpy as np
import re

In [3]:
_filepath = r"C:\Users\marcu\OneDrive\NaluScientific\obsidian_notes\NaluNote\Day Planners"

In [4]:
def get_todays_file():
    today = datetime.now()
    FILENAME = f"Day Planner-{today.strftime('%Y%m%d')}.md"
    FILEPATH = Path(r"C:\Users\marcu\OneDrive\NaluScientific\obsidian_notes\NaluNote\Day Planners")
    return FILEPATH / FILENAME

In [5]:
def count_workdays(start_date: datetime, end_date: datetime):
    """Count the workdays between two dates"""
    return np.busday_count(start_date.strftime('%Y-%m-%d'), end_date.strftime('%Y-%m-%d'))

In [6]:
from typing import List, Tuple

def scrape_time_and_comments_from_md(file_path: pathlib.Path) -> List[List[Tuple[str]]]:
    """Scrape the contents of a file into a list.

    This function assumes the file contains the timelog in to following pattern:
    - [ ] 9:00 project | comment
    - 10:00 comment

    If no separator is found, then it's assumed there's no project attached.
    If no project is attached, lave blank.

    """
    # First pattern to extract time and the combined comment and project part
    first_pattern = r'\[?.?]?\s*(\d{2}:\d{2})\s+(.+?)\n'

    with open(file_path, 'r') as file:
        content = file.readlines()

    matches = re.findall(first_pattern, ''.join(content))

    second_pattern = r'(.*?)\s*\|\s*(.*)'  # Used to identify the separator
    separator= '|'
    final_matches = []
    for time, comment_project in matches:
        if separator in comment_project:
            split_match = re.match(second_pattern, comment_project)
            if split_match:
                project, comment = split_match.groups()
            else:
                project, comment = '', ''
        else:
            project, comment = '', comment_project

        final_matches.append((time, "", "", project.strip(), comment.strip()))

    return final_matches

In [7]:
def scrape_time_and_comments_from_md(file_path):

    with open(file_path, 'r') as file:
        content = file.readlines()
    first_pattern = r'\[?.?]?\s*(\d{2}:\d{2})\s+(.+?)\n'
    matches = re.findall(first_pattern, ''.join(content))

    second_pattern = r'(.*?)\s+\|\s+(.*)'
    final_matches = []
    for time, comment_project in matches:
        if '|' in comment_project:
            # Split comment and project
            project, comment = re.match(second_pattern, comment_project).groups()
        else:
            # No project, comment takes the whole part
            comment, project = comment_project, ''

        # Add to final matches
        final_matches.append((time, "", "", project.strip(), comment.strip()))

    return final_matches

In [8]:
def scrape_time_and_comments_from_md(file_path):
    # Regular expression to match the required pattern
    # Adjusted to handle the newline character correctly
    # pattern = r'- \[.\] (\d{2}:\d{2}) (.+?)\n'
    # pattern = r'\[?.?]?\s*(\d{2}:\d{2})\s+(.+?)\n'
    pattern = r'-?\[?.?]?\s*(\d{2}:\d{2})\s+(.*?)(?:\s+\|\s+(.*?))?\n'

    # Read the file
    with open(file_path, 'r') as file:
        content = file.readlines()

    # Extract matches
    matches = re.findall(pattern, ''.join(content))
    return matches

In [9]:
# Scrape the time and comments from the temporary file again with the adjusted pattern
scraped_data = scrape_time_and_comments_from_md(FULLPATH)
scraped_data

NameError: name 'FULLPATH' is not defined

In [10]:
# import gspread
# from oauth2client.service_account import ServiceAccountCredentials

# Format the data into the required structure
def format_data(data):
    formatted_data = []
    for time, comment in data:
        date_of_file = datetime.now().strftime("%Y-%m-%d")  # Replace with actual date if needed
        amount_since_last_row, sum, amount_since_last_row_copy = calculate_additional_data((time, comment))
        formatted_row = [date_of_file, time, amount_since_last_row, sum, "", comment, amount_since_last_row_copy]
        formatted_data.append(formatted_row)
    return formatted_data

# Use creds to create a client to interact with the Google Drive API
# scope = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive']
# creds = ServiceAccountCredentials.from_json_keyfile_name('path_to_your_credentials_file.json', scope)
# client = gspread.authorize(creds)

# Open the sheet and select the first sheet
# sheet = client.open("Your_Sheet_Name").sheet1

sheet = []

# Insert a blank row
# sheet.append_row([])

# Extracted data
data_to_insert = scraped_data  # Use the data you extracted earlier
formatted_data = format_data(data_to_insert)

# Append formatted data to the sheet
for row in formatted_data:
    # sheet.append_row(row)
    sheet.append(row)


NameError: name 'scraped_data' is not defined

In [11]:
from datetime import datetime

def calc_sum(timestamp1, timestamp2, row_description1, row_description2):
    # Check conditions
    if row_description1.upper() in ["BREAK", "END", ""]:
        return 0.0
    # if row_description2 in ["BREAK", "END", ""]:
        # return 0.0

    # Parse timestamps and calculate the difference in hours
    format = "%H:%M" # %H:%M"  # Adjust the format if necessary
    time1 = datetime.strptime(timestamp1, format)
    time2 = datetime.strptime(timestamp2, format)
    # print(f"{time1=}, {time2=}")
    time_diff_hours = (time2 - time1).total_seconds() / 3600

    return time_diff_hours

def calc_total_sum(scraped_data):
    tot_sum = 0
    
    for i in range(1, len(scraped_data)):
        prev_row = scraped_data[i-1]
        row = scraped_data[i]
        ts1 = prev_row[0]
        ts2 = row[0]
        desc1 = prev_row[-1]
        desc2 = row[-1]
        sum_value = calc_sum(ts1, ts2, desc1, desc2)
        tot_sum += sum_value
    return tot_sum

calc_total_sum(scraped_data)

NameError: name 'scraped_data' is not defined

In [12]:
def calc_daysum(filename):
    scraped_data = scrape_time_and_comments_from_md(filename)
    totsum = calc_total_sum(scraped_data)
    return totsum

In [18]:
_filepath = get_todays_file()
calc_daysum(_filepath)

5.25

In [19]:
def calc_weeksum(folder, weekstart, numdays):
    results = 0
    for day in range(numdays):
        target_date = weekstart + relativedelta(days=+day)
        filename = "Day Planner-{}.md".format(target_date.strftime('%Y%m%d'))
        fullpath = folder / filename
        try:
            h = calc_daysum(fullpath)
        except Exception as e:
            # print(e)
            h = 0.0
        results += h
    return results

In [21]:
calc_weeksum(_filepath, datetime.fromisoformat("20240116"), 14)

0.0

In [24]:
from typing import Literal

def calc_timeperiod(folder, year: int, month: int, period: Literal["first", "second", "both"]):
    if not 0 < month <= 12:
        raise ValueError("Only 12 months in a year")
    start, end = get_timeperiod_or_raise(year, month, period)
    print(f"Calculating period: {start:%F} - {end:%F}")
    curr = start
    h_tot = 0

    while curr <= end:
        filename = "Day Planner-{}.md".format(curr.strftime('%Y%m%d'))
        fullpath = folder / filename
        try:
            h = calc_daysum(fullpath)
        except:
            h = 0.0
        h_tot += h
        curr += relativedelta(days=+1)
    return h_tot

calc_timeperiod(_filepath, 2024, 1, "second")

NameError: name 'get_timeperiod_or_raise' is not defined

In [25]:
def get_timeperiod_or_raise(year: int, month: int, period: Literal["first", "second", "both"]):
    """Retuns the first and last date of the payperiod.

    Assumes a bimonthly payment plan where the divider is the 15th of the month.

    Args:
        year: 4 digit year
        month: 1-12
        period: first, second or both

    Returns:
        first and last date of the timeperiod. If both is selected, will return first and last date of month.
    """
    first_start = datetime.fromisoformat(f"{year}{month:02}01")
    tp_month = first_start + relativedelta(months=+1)
    first_end = first_start + relativedelta(days=+14)
    second_start = first_end + relativedelta(days=+1)
    second_end = tp_month + relativedelta(days=-1)
    if period == "first":
        start = first_start
        end = first_end
    elif period == "second":
        start = second_start
        end = second_end
    elif period == "both":
        start = first_start
        end = second_end
    else:
        raise ValueError("Period must be 'first' or 'second'")
    return start, end

In [26]:
def create_date_axis(start: datetime, end: datetime):
    axis = []
    curr = start
    while curr <= end:
        axis.append(curr)
        curr += relativedelta(days=1)
    return axis

In [27]:
def get_period_dividers(year, month, period: Literal["first", "second", "both"]):
    """Returns the two time periods for a given month as tuples.

    Args:
        year: 4 digit year.
        month: month number 1 - 12

    Returns:
        Tuple ((first_start, first_end), (second_start, second_end))
    """
    if not isinstance(year, int):
        raise TypeError("yeah must be an int")
    if len(str(year)) != 4:
        ValueError("Year must be 4 digits")
    if not 0 < month <= 12:
        raise ValueError("Only 12 months in a year")
    first_start = datetime.fromisoformat(f"{year}{month:02}01")
    tp_month = first_start + relativedelta(months=+1)
    
    first_end = first_start + relativedelta(days=+14)
    second_start = first_end + relativedelta(days=+1)
    second_end = tp_month + relativedelta(days=-1)

    return ((first_start, first_end), (second_start, second_end))

In [28]:
def gen_copy_paste(folder: pathlib.Path, year: int, month: int, period: Literal["first", "second", "both"]):
    if not 0 < month <= 12:
        raise ValueError("Only 12 months in a year")
    start, end = get_timeperiod_or_raise(year, month, period)

    curr = start
    timesheet = []
    while curr <= end:
        filename = "Day Planner-{}.md".format(curr.strftime('%Y%m%d'))
        fullpath = folder / filename
        try:
            scraped = scrape_time_and_comments_from_md(fullpath)
        except Exception as e:
            scraped = []
            pass
        row = [(curr.strftime('%Y-%m-%d'), *item) for item in scraped]
        if row:
            timesheet.extend(row)
            timesheet.append(None)
        curr += relativedelta(days=+1)
    for row in timesheet:
        if row is None:
            print("")
            continue
        print(f"{', '.join(row)}")

In [30]:
gen_copy_paste(_filepath, 2024, 1, "second")

In [50]:
count_workdays(*pf)

start_date=datetime.datetime(2024, 1, 1, 0, 0), end_date=datetime.datetime(2024, 1, 15, 0, 0)


10

In [51]:
_today = datetime.now()

count_workdays(_today, pf[1])

start_date=datetime.datetime(2024, 1, 10, 20, 58, 44, 972628), end_date=datetime.datetime(2024, 1, 15, 0, 0)


3

In [52]:
count_workdays(pf[0], _today)

start_date=datetime.datetime(2024, 1, 1, 0, 0), end_date=datetime.datetime(2024, 1, 10, 20, 58, 44, 972628)


7

In [ ]:
'date', 'start_time', 'amount, buildup, project, comment 